Import library yang akan digunakan

In [1]:
import numpy as np 
import pandas as pd 
import os
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

Kemudian, lakukan mount drive dengan code berikut

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Membaca dataset yang kita miliki

In [3]:
movie = pd.read_csv('/content/drive/MyDrive/Dataset/MovieLens/movie.csv')
rating= pd.read_csv('/content/drive/MyDrive/Dataset/MovieLens/rating.csv')

Pratinjau data

In [4]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


Membuat kolom Tahun di movie dataframe

In [6]:
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False)

Menghapus tahun di kolom Judul.

In [7]:
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
movie['title'] = movie['title'].apply(lambda x: x.strip())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
movie.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Menghapus kolom genre

In [9]:
movie.drop(columns=['genres'], inplace=True)
movie.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Menghapus kolom timestamp pada dataframe rating

In [10]:
rating.drop(columns=['timestamp'], inplace=True)

In [11]:
rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


Proses pembuatan sistem rekomendasi Berbasis User adalah sebagai berikut:
*   Pilih pengguna dengan film yang telah ditonton pengguna
*   Mendapatkan rekaman film yang ditonton dari pengguna
*   Menghitung similarity score
*   Merekomendasikan item dengan skor tertinggi









In [12]:
user = [
            {'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            {'title':'Jumanji', 'rating':3},
            {'title':"Pulp Fiction", 'rating':4.5},
            {'title':'Akira', 'rating':5}
         ] 
inputMovie = pd.DataFrame(user)
inputMovie

,title,rating
0,"Breakfast Club, The",4.0
1,Toy Story,2.5
2,Jumanji,3.0
3,Pulp Fiction,4.5
4,Akira,5.0


Memfilter film berdasarkan judul, Kemudian gabungkan, Sehingga kita bisa mendapatkan movieId. Menghapus informasi yang tidak digunakan 

In [13]:
Id = movie[movie['title'].isin(inputMovie['title'].tolist())]
inputMovie = pd.merge(Id, inputMovie)
inputMovie = inputMovie.drop('year', 1)
inputMovie

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,rating
0,1,Toy Story,2.5
1,2,Jumanji,3.0
2,296,Pulp Fiction,4.5
3,1274,Akira,5.0
4,1968,"Breakfast Club, The",4.0


Memfilter pengguna yang telah menonton film yang telah ditonton oleh input dan menyimpannya

In [14]:
users = rating[rating['movieId'].isin(inputMovie['movieId'].tolist())]
users.head()

,userId,movieId,rating
0,1,2,3.5
11,1,296,4.0
236,3,1,4.0
451,5,2,3.0
517,6,1,5.0


In [15]:
users.shape

(168730, 3)

Membuat beberapa sub dataframes di mana semuanya memiliki nilai yang sama di kolom yang ditentukan sebagai parameter

In [16]:
userSubsetGroup = users.groupby(['userId'])

Menunjukkan satu contoh grup dengan mendapatkan semua pengguna uderId tertentu

In [17]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
166633,1130,1968,4.0


Menyortirnya sehingga pengguna dengan film yang paling mirip dengan input akan diprioritaskan

In [18]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [19]:
userSubsetGroup[0:3]

[(91,       userId  movieId  rating
  9621      91        1     4.0
  9622      91        2     3.5
  9669      91      296     3.5
  9826      91     1274     2.5
  9903      91     1968     4.0), (294,        userId  movieId  rating
  37452     294        1     4.5
  37453     294        2     4.5
  37504     294      296     4.5
  37648     294     1274     4.5
  37731     294     1968     5.0), (586,        userId  movieId  rating
  81164     586        1     2.5
  81165     586        2     3.0
  81226     586      296     5.0
  81390     586     1274     4.0
  81499     586     1968     3.0)]

In [20]:
userSubsetGroup = userSubsetGroup[0:100]

* Simpan Pearson Correlation dalam dictionary, di mana keynya user Id pengguna dan value adalah coefficient
* Menyortir input dan user group saat ini sehingga nilainya tidak tercampur nanti
* Mendapatkan review scores untuk film 
* Simpan dalam variabel buffer sementara dalam untuk memfasilitasi perhitungan di masa mendatang
* Letakkan ulasan grup pengguna saat ini dalam format list
* Hitung pearson correlation antara dua pengguna, yang disebut, x dan y





In [21]:
pearsonCorDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    inputMovie = inputMovie.sort_values(by='movieId')
    n = len(group)
    temp = inputMovie[inputMovie['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)

    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0

In [22]:
pearsonCorDict.items()

dict_items([(91, -0.6890618270883883), (294, 0.10783277320343156), (586, 0.7836445860269199), (648, 0.444102681159703), (775, 0.46266531814837414), (812, -0.12945217625467967), (869, 0.07624928516630236), (903, 0.0660338179744212), (1200, 0.2494610901255917), (1244, 0.29654012630945475), (1715, 0.6309898162000303), (1748, 0.5114957546028552), (1763, 0.1760901812651271), (1810, 0.6990252954195334), (1813, 0.36589645615870564), (1849, 0.06603381797442423), (1864, 0.5114957546028552), (1942, 0.23262521394079627), (1984, -0.7994259492812168), (2047, 0.5477103564747346), (2099, -0.10783277320343156), (2367, -0.10783277320343994), (2397, 0), (2515, 0.9244734516419062), (2661, 0.835703992326648), (2757, 0.8439249387982215), (2959, 0.23055616708169688), (2988, 0.29809064964264287), (3179, 0.0), (3218, 0.26413527189768793), (3268, 0.7781270639007126), (3269, 0.3606167767094639), (3318, 0.3026049692947228), (3397, 0.46107317554294897), (3487, -0.3774147062120338), (3576, 0.39620290784652895), (3

In [23]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.689062,91
1,0.107833,294
2,0.783645,586
3,0.444103,648
4,0.462665,775


In [24]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
77,0.964486,9305
80,0.951044,9650
63,0.940064,8032
81,0.924473,9772
23,0.924473,2515


Mengambil weighted average dari peringkat film menggunakan Pearson Correlation sebagai bobot. Tetapi untuk melakukan ini, Dapatkan film yang ditonton oleh pengguna di pearsonDF kami dari ratings dataframe dan kemudian menyimpan korelasinya di kolom baru yang disebut _similarityIndex".

In [25]:
topUsersRating=topUsers.merge(rating, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.964486,9305,1,1.0
1,0.964486,9305,2,1.0
2,0.964486,9305,11,1.0
3,0.964486,9305,21,1.0
4,0.964486,9305,32,2.0


Mengalikan similarity dengan user's ratings

In [26]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.964486,9305,1,1.0,0.964486
1,0.964486,9305,2,1.0,0.964486
2,0.964486,9305,11,1.0,0.964486
3,0.964486,9305,21,1.0,0.964486
4,0.964486,9305,32,2.0,1.928971


Menerapkan sum ke topUsers setelah mengelompokkannya berdasarkan userId

In [27]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,30.998297,105.356196
2,30.998297,85.222713
3,9.567244,21.072345
4,0.888910,2.289317
5,5.368970,13.468631


Membuat empty dataframe, Terus mengambil weighted average



In [28]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.398774,1
2,2.749271,2
3,2.202551,3
4,2.575419,4
5,2.508606,5


In [29]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
4422,5.0,4422
25744,5.0,25744
42217,5.0,42217
2068,5.0,2068
1419,5.0,1419
8785,5.0,8785
93164,5.0,93164
26587,5.0,26587
1585,5.0,1585


Mendapatkan 10 film teratas dengan skor rekomendasi tertinggi. Ini adalah 10 rekomendasi teratas untuk pengguna input berdasarkan apa yang ditonton orang lain

In [30]:
movie.loc[movie['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
1385,1419,Walkabout,1971
1533,1585,Love Serenade,1996
1984,2068,Fanny and Alexander (Fanny och Alexander),1982
4327,4422,Cries and Whispers (Viskningar och rop),1972
5311,5408,Nora,2000
8102,8785,Early Summer (Bakushû),1951
8345,25744,Haxan: Witchcraft Through the Ages (a.k.a. The...,1922
8937,26587,"Decalogue, The (Dekalog)",1989
10709,42217,Late Spring (Banshun),1949
18706,93164,Sleep Tight (Mientras duermes),2011
